In [1]:
#Iniciando modulos
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import pandas as pd
import pyodbc
import time
import re

#Acesso ao banco de dados Trab
def executar_conexao_bd():
    server = 'TBDBL-GLEISSON'
    database = 'Dados_Pavimento'
    username = 'gleisson.santos'
    password = '12345'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

Executado a conexão com o BD......


In [ ]:
#Acesso ao banco de dados Casa
def executar_conexao_bd():
    server = 'DESKTOP-0P0H9IO'
    database = 'Dados_Pavimento'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

In [308]:
#df = pd.read_sql('SELECT * FROM [Hds700]', executar_conexao_bd())
df = pd.read_sql('SELECT * FROM Hds700', executar_conexao_bd())
df = pd.DataFrame(df, columns = ['SS', 'OS','Tipo', 'Serviço','Ano'])
#df = df.astype(str)

# Filtra as linhas onde "Tipo" é igual a "26 - MANUTENCAO DE HIDROMETRO" e "Serviço" é igual a "12 - SUBSTITUICAO HID." OU "766 - REV HIDROMETRO PARADO"
#df = df.loc[(df['Tipo'] == '26 - MANUTENCAO DE HIDROMETRO') & ((df['Serviço'] == '12 - SUBSTITUICAO HID.') | (df['Serviço'] == '766 - REV HIDROMETRO PARADO') | (df['Serviço'] == '194 - SUBST HD VOLUMETRICO'))]
df = df.loc[(df['Tipo'] == '26 - MANUTENCAO DE HIDROMETRO') & ((df['Serviço'] == '12 - SUBSTITUICAO HID.') | (df['Serviço'] == '194 - SUBST HD VOLUMETRICO'))]

df = df.iloc[170:500, :]


# Seleciona apenas a coluna "Serviço"
#df = df.head(2)

display(df)
n_linhas = len(df)
print("Número de linhas: ", n_linhas)



Executado a conexão com o BD......


,SS,OS,Tipo,Serviço,Ano
8992,945274329,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020
8993,948721677,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020
8994,951342447,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020
8995,951957326,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020
8996,951957330,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2020
...,...,...,...,...,...
10299,963328598,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2021
10300,963328615,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2021
10301,962738402,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2021
10302,963148475,1,26 - MANUTENCAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,2021


Número de linhas:  452


In [212]:
#Inicinado automação / abrindo navegador
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()
time.sleep(1)

login = 't034183'
senha = 'CNB@2022'

driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
time.sleep(1)

# Procurar a tag random do site / Login e senha
randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
driver.find_element(By.ID, "loginForm-submit").click()
time.sleep(1)
arvore_acesso(driver, 'insum', 'GOSI_anchor')

time.sleep(0.5)
driver.execute_script("window.open('https://sciweb.embasanet.ba.gov.br/');")
driver.switch_to.window(driver.window_handles[1])
time.sleep(0.5)
arvore_acesso(driver, 'detalhe', 'DOSE_anchor')
driver.switch_to.window(driver.window_handles[0])
driver.switch_to.frame("frame-content")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - Driver [C:\Users\gleisson.santos\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache
C:\Users\GLEISS~1.SAN\AppData\Local\Temp/ipykernel_7672/2965849731.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [34]:
#tipo: insu, menu: GOSI_anchor - ACESSAR MENU DE INSUMOS
def arvore_acesso(driver, tipo, menu ):
    # Proc detalhe da Os no campo de buscar
    driver.find_element(By.ID, 'arvoreSearch').send_keys(tipo)
    # aguardar ate o elemento aparecer e/ou seguir codigo
    wait = WebDriverWait(driver, 10)
    element1 = wait.until(EC.element_to_be_clickable((By.ID, menu)))
    # acessar Campo detalhe da Os
    driver.find_element(By.ID, menu).click()
    time.sleep(1)
    #acessar camada frame do site
    driver.switch_to.frame("frame-content")


In [41]:
arvore_acesso(driver, 'insum', 'GOSI_anchor')

In [109]:
#Consultar na label Material
def preencher_ss(driver, lista, lista2):
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt72-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt80-item"]').clear()
    
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt72-item"]').send_keys(lista)
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt80-item"]').send_keys(lista2)
    time.sleep(0.5)
    # clinical em buscar
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-panel"]/input').click()


In [213]:
preencher_ss(driver, '902766338', '1')

In [196]:
#Consultar na label Material
def preencher_os(driver, lista, lista2):
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').clear()
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').send_keys(lista)
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').send_keys(lista2)
    time.sleep(0.5)
    # clinical em buscar
    driver.find_element(By.XPATH, '//*[@name="form-j_idt70-j_idt90"]').click()
    # Coletar HD 
    time.sleep(0.5)
    hd = driver.find_element(By.XPATH, "//*[@id='form-j_idt391-j_idt392-item']")
    text = hd.text
    return text


In [197]:
def buscar_hd(driver):
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame("frame-content")
    hd = preencher_os(driver, '902766338', '1')
    time.sleep(1)
    driver.switch_to.window(driver.window_handles[0])
    driver.switch_to.frame("frame-content")
    return hd

In [198]:
buscar_hd(driver)

'A19N363580'

In [202]:
um_meio = ['INTDIG01 - HIDROMETRO RES MONO 0,75M3/H 1/2"S/C 45',           
           'INTDIG01 - CONJ CONEXOES HIDROM 1/2 PP 1,5 M3/H',           
           'INTDIG01 - REGISTRO PP 1/2" ACIONAM RESTRITO PN10',
          'INTDIG01 - LACRE TAMPA CX HIDROM AJUSTAVEL (AZUL)']

de_tres = ['INTDIG01 - HIDROMETRO RES MULT 1,5M3/H 1/2" S/C 45º',           
           'INTDIG01 - CONJ CONEXOES HIDROM 1/2 PP 3 E 1,5 M3/H',           
          'INTDIG01 - REGISTRO PP 1/2" ACIONAM RESTRITO PN10',
          'INTDIG01 - LACRE TAMPA CX HIDROM AJUSTAVEL (AZUL)']

In [203]:
def campo_material(driver, material1, material2):
    texto = buscar_hd(driver)
    if not isinstance(texto, str):
        texto = str(texto)
        
    padrao_ay = r"[A-Z][0-9][0-9][A-Z]"
    resultados = re.findall(padrao_ay, texto)

    try:
        campo1 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="form-cpnlAssociarInsumo-tmpCodigoAutoComp-codAutoComp_input"]')))
    except NoSuchElementException:
        print("Erro na Unidade de medição")
        return
    
    try:
        if any(resultado.startswith('A') for resultado in resultados):
            campo1.click()
            time.sleep(1)
            campo1.clear()
            time.sleep(1)
            campo1.send_keys(material1)
            time.sleep(1)
            driver.find_element(By.XPATH,'//*[@class="ui-autocomplete-query"]').click()   

        elif any(resultado.startswith('Y') for resultado in resultados):
            campo1.click()
            time.sleep(1)
            campo1.clear()
            time.sleep(1)
            campo1.send_keys(material2)
            time.sleep(1)
            driver.find_element(By.XPATH,'//*[@class="ui-autocomplete-query"]').click() 
        else:
            print("Hd incorreto")
    except NoSuchElementException:
        print("Não foi encontrado Hd")



In [204]:
campo_material(driver, de_tres, um_meio)

Não foi encontrado Hd


In [214]:
#Acessando e inserindo quantidade e medir
def campo_quantidade(driver, qtd, medir):
    try:
        campo2 = driver.find_element(By.XPATH, '//*[@id="form-cpnlAssociarInsumo-quantidade-item"]')
        campo2.click()
        campo2.clear()
        campo2.send_keys(qtd)
        time.sleep(0.5)
        # Acessando e inserindo Disp p medir S/N
        select_element = driver.find_element(By.ID, 'form-cpnlAssociarInsumo-j_idt290-comboBooleanSN-item')
        select = Select(select_element)
        select.select_by_value(medir)
        # select.select_by_visible_text('Não')
        time.sleep(0.5)
        # Confirar insumo
        driver.find_element(By.XPATH,'//*[@id="form-cpnlAssociarInsumo-confirmar"]').click()
    except NoSuchElementException:
        print("Elemento não encontrado. Verifique o seletor XPATH do elemento.")

    


In [206]:
campo_quantidade(driver, '1', 'N')

In [309]:
um_meio = ['INTDIG01 - HIDROMETRO RES MONO 0,75M3/H 1/2"S/C 45',           
           'INTDIG01 - CONJ CONEXOES HIDROM 1/2 PP 1,5 M3/H',           
           'INTDIG01 - REGISTRO PP 1/2" ACIONAM RESTRITO PN10',
          'INTDIG01 - LACRE TAMPA CX HIDROM AJUSTAVEL (AZUL)']

de_tres = ['INTDIG01 - HIDROMETRO RES MULT 1,5M3/H 1/2" S/C 45º',           
           'INTDIG01 - CONJ CONEXOES HIDROM 1/2 PP 3 E 1,5 M3/H',           
          'INTDIG01 - REGISTRO PP 1/2" ACIONAM RESTRITO PN10',
          'INTDIG01 - LACRE TAMPA CX HIDROM AJUSTAVEL (AZUL)']

#Consultar e buscar HD
def preencher_os(driver, lista, lista2):
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').clear()
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').send_keys(lista)
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').send_keys(lista2)
    time.sleep(0.5)
    # clinical em buscar
    driver.find_element(By.XPATH, '//*[@name="form-j_idt70-j_idt90"]').click()
    # Coletar HD 
    time.sleep(0.5)
    hd = driver.find_element(By.XPATH, "//*[@id='form-j_idt391-j_idt392-item']")
    text = hd.text
    time.sleep(1)
    driver.switch_to.window(driver.window_handles[0])
    driver.switch_to.frame("frame-content")
    return text

#Alterar pagina e pegar  numero do HD
def buscar_hd(driver):
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame("frame-content")

def campo_material(driver, material1, material2, texto):   
    #texto = 'A15S0015675'
   
    padrao_ay = r"[A-Z][0-9][0-9][A-Z]"
    resultados = re.findall(padrao_ay, texto)

    try:
        campo1 = driver.find_element(By.XPATH,'//*[@id="form-cpnlAssociarInsumo-tmpCodigoAutoComp-codAutoComp_input"]')
    except NoSuchElementException:
        print("Erro na Unidade de medição")
        pass
    
    try:
        if any(resultado.startswith('A') for resultado in resultados):
            time.sleep(1)
            campo1.click()
            time.sleep(1)
            campo1.clear()
            time.sleep(1)
            campo1.send_keys(material1)
            time.sleep(2)
            driver.find_element(By.XPATH,'//*[@class="ui-autocomplete-query"]').click()   


        elif any(resultado.startswith('Y') for resultado in resultados):
            time.sleep(1)
            campo1.click()
            time.sleep(1)
            campo1.clear()
            time.sleep(1)
            campo1.send_keys(material2)
            time.sleep(2)
            driver.find_element(By.XPATH,'//*[@class="ui-autocomplete-query"]').click() 
        else:
            print("Hd incorreto")
    except NoSuchElementException:
        print("Não foi encontrado Hd")



#Acessando e inserindo quantidade e medir
def campo_quantidade(driver, qtd, medir):
    try:
        campo2 = driver.find_element(By.XPATH, '//*[@id="form-cpnlAssociarInsumo-quantidade-item"]')
        campo2.click()
        campo2.clear()
        time.sleep(0.5)
        campo2.send_keys(qtd)
        time.sleep(0.5)
        # Acessando e inserindo Disp p medir S/N
        select_element = driver.find_element(By.ID, 'form-cpnlAssociarInsumo-j_idt290-comboBooleanSN-item')
        select = Select(select_element)
        select.select_by_value(medir)
        # select.select_by_visible_text('Não')
        time.sleep(0.5)
        # Confirar insumo
        driver.find_element(By.XPATH,'//*[@id="form-cpnlAssociarInsumo-confirmar"]').click()
    except NoSuchElementException:
        print("Elemento não encontrado. Verifique o seletor XPATH do elemento.")


def esperar_sumir(driver):
    wait = WebDriverWait(driver, 30)
    element = wait.until(EC.invisibility_of_element_located((By.ID, "j_idt24_modal")))

def geral2():
    #Inicinado automação / abrindo navegador
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    chrome_options = webdriver.ChromeOptions()
    time.sleep(1)

    login = 't034183'
    senha = 'CNB@2022'

    driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
    time.sleep(1)

    # Procurar a tag random do site / Login e senha
    randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
    driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
    driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
    driver.find_element(By.ID, "loginForm-submit").click()
    time.sleep(1)
    arvore_acesso(driver, 'insum', 'GOSI_anchor')

    time.sleep(0.5)
    driver.execute_script("window.open('https://sciweb.embasanet.ba.gov.br/');")
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(0.5)
    arvore_acesso(driver, 'detalhe', 'DOSE_anchor')
    driver.switch_to.window(driver.window_handles[0])
    driver.switch_to.frame("frame-content")

    contador = 0

    for i, row in df.iterrows():
        preencher_ss(driver, str(row['SS']), str(row['OS']))
        time.sleep(1)
        buscar_hd(driver)
        time.sleep(0.5)
        try:
            resultado_os = preencher_os(driver, str(row['SS']), str(row['OS']))
            time.sleep(0.5)
            for material1, material2 in zip(de_tres, um_meio):
                campo_material(driver, material1, material2, resultado_os)
                time.sleep(0.5)
                esperar_sumir(driver)
                campo_quantidade(driver, 1, 'N')
                time.sleep(1)
            contador += 1
            from IPython.display import clear_output
            clear_output(wait=True)
            print(f"{contador} Ss(s) insumada(s) com sucesso!")
        except:
            pass





In [ ]:
geral2()

112 Ss(s) insumada(s) com sucesso!
